In [1]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col


spark = SparkSession \
        .builder \
        .config("spark.jars",
                "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
        .master("local[*]") \
        .appName("leetcode-problems") \
        .getOrCreate()


24/06/06 20:40:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
employees = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/credit_card") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "employees") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

In [11]:
from pyspark.sql.functions import count,lower,when
from pyspark.sql.window import Window

employees\
    .withColumn("lower_email",lower("email_id"))\
    .withColumn("same",when(col("email_id") == col("lower_email"),'Yes').otherwise("No"))\
    .withColumn("cnt",count("lower_email").over(Window.partitionBy(col("lower_email"))))\
    .filter((col("same") == "Yes") & (col("cnt")>1))\
    .drop("same","cnt","lower_email")\
    .show()

+-----------+-------------+-------------+
|employee_id|employee_name|     email_id|
+-----------+-------------+-------------+
|        102|     Josh Day|jo.da@abc.com|
|        105|   Toby Scott|jo.da@abc.com|
+-----------+-------------+-------------+

